# iMessage data preparation

In [ ]:
import sqlite3
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline


Set constants

In [ ]:
IMESSAGE_DB = '/Users/home/Library/Messages/chat.db'
CONTACTS = {
    'juliadolzhikova@gmail.com':'Julia',
    '+420739844387':'Julia',
    '+48577345096':'Julia',
    'julia.shablya@icloud.com':'Julia',

    'shablyairyna@gmail.com':'Mama',

    '+393402103517':'Alia',

    'v.stebaeva@icloud.com':'Babulia',
    'v.stebaeva@gmail.com':'Babulia'
}

Create connection to iMessage DB

In [ ]:
conn = sqlite3.connect(IMESSAGE_DB)

In [177]:
contacts = pd.DataFrame([['juliadolzhikova@gmail.com', 'shablyairyna@gmail.com', '+393402103517', 'v.stebaeva@icloud.com'],
    ['+420739844387', '', '', 'v.stebaeva@gmail.com'],
    ['+48577345096', '', '', ''],
    ['julia.shablya@icloud.com', '', '', '']],
 columns=["Julia", "Mama", "Alia", "Babulia"])
# contacts.to_csv('../Data/contacts.csv')
n = contacts.columns[contacts.eq('v.stebaeva@gmail.com').any()][0]
n

'Babulia'

Read contacts and define function for getting contact name by number

In [182]:
contacts = pd.read_csv('../Data/contacts.csv')

def getContact(account:str):
    name = contacts.columns[contacts.eq(account).any()]
    if len(name) == 0:
        return "Other"
    else:
        return name[0]

Get messages table and convert date to human readable format

In [ ]:
messages = pd.read_sql_query('''select *, datetime(date/1000000000 + strftime("%s", "2001-01-01") ,"unixepoch","localtime")  as timestamp from message''', conn) 
messages.rename(columns={'ROWID' : 'message_id'}, inplace = True)
messages['timestamp'] = messages['timestamp'].apply(lambda x: pd.Timestamp(x))
messages['date'] = messages['timestamp'].apply(lambda x: x.date())
messages = messages[['message_id', 'handle_id', 'text', 'is_sent', 'timestamp', 'date']]
messages.head()

get the handles to apple-id mapping table

In [ ]:
handles = pd.read_sql_query("select ROWID as handle_id, id as account, service  from handle", conn)
handles.head()

Merge messages with handles

In [ ]:
message_to_handle = pd.merge(messages[['message_id', 'handle_id', 'text', 'is_sent', 'timestamp', 'date']],  handles[['handle_id', 'account', 'service']], on ='handle_id', how='left')
message_to_handle.head(10)

Get the chat to message mapping and join together with message_to_handle

In [ ]:
chat_message_joins = pd.read_sql_query("select * from chat_message_join", conn)

data = pd.merge(message_to_handle, chat_message_joins[['chat_id', 'message_id']], on = 'message_id', how='left')
data.head(10)

Map each handle to particular sender

In [ ]:
data.handle_id = data.account.map(lambda x: CONTACTS.get(x, 'Other'))
data.rename(columns={'handle_id' : 'sender'}, inplace = True)
data.head()

Extract information about attachments

In [ ]:
message_attachment_joins = pd.read_sql_query("select * from message_attachment_join", conn)
attachments = pd.read_sql_query("select ROWID as attachment_id, filename from attachment", conn)
attachment_per_message = pd.merge(message_attachment_joins,  attachments, on ='attachment_id', how='left')
attachment_per_message

Add attachment path to data df

In [ ]:
data = pd.merge(data, attachment_per_message[['message_id', 'filename']], how='left', on='message_id')
data.head()

Rearange df columns

In [ ]:
data = data[['message_id', 'chat_id', 'sender', 'account', 'service', 'is_sent', 'text', 'timestamp', 'date', 'filename']]
data.head()

Visualize information

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
data['sender'].value_counts().plot(kind='pie')

In [ ]:
data['service'].value_counts().plot(kind='pie')

In [ ]:
data['is_sent'].value_counts().plot(kind='pie')

In [ ]:
data['date'].value_counts().plot(kind='line')

Save results to CSV file

In [ ]:
data.to_csv('~/Downloads/iMessage.csv', index = False, encoding='utf-16')

# Display information in chat-style format

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

Read data from CSV

In [ ]:
imessage = pd.read_csv('~/Downloads/iMessage.csv', encoding='utf-16', na_values=['', ' ', '\n', '\t', '\r\n', '\r', '￼'])
imessage['timestamp'] = imessage['timestamp'].apply(lambda x: pd.Timestamp(x))
imessage['text'] = imessage['text'].apply(lambda x: str(x))
imessage.info()

Cleanup

In [ ]:
imessage.drop(imessage[imessage.text.eq('nan')].index, axis=0, inplace=True)
imessage.drop(imessage[imessage.text.str.startswith('http')].index, axis=0, inplace=True)
imessage.drop(imessage[imessage.text.str.startswith('Понравилось «')].index, axis=0, inplace=True)
imessage.drop(imessage[imessage.text.str.isspace()].index, axis=0, inplace=True)
imessage.head(20)

Difine function for getting sender specific vocabulary

In [ ]:
import re
from collections import Counter

def getVocabulary(sender: str = 'Me', isSent=0):
    if sender == 'Me':
        senderMessages = imessage[imessage['is_sent'] == 1].text.values
    else:
        senderMessages = imessage[imessage.sender.eq(sender) & imessage.is_sent.eq(isSent)].text.values

    vocabulary = []
    pattern = re.compile(f'[А-Яа-я]+')
    for s in senderMessages:
        for word in s.split():
            word = word.lower().replace('.','').replace(',','').replace('!','').replace('?','').strip()
            if pattern.match(word) and len(word)>1:
                vocabulary.append(word)
    return Counter(vocabulary)

Vocabulary visualization

In [ ]:
myVocabulary = v = getVocabulary()
words, counts = zip(*myVocabulary.most_common(20))
plt.barh(words, counts)

In [ ]:
juliaVocabulary = v = getVocabulary('Julia')
words, counts = zip(*juliaVocabulary.most_common(20))
plt.barh(words, counts)

In [ ]:
mamaVocabulary = v = getVocabulary('Mama')
words, counts = zip(*mamaVocabulary.most_common(20))
plt.barh(words, counts)

Extract chat with Julia

In [ ]:
julia = imessage[imessage['sender'] == 'Julia'][['text', 'is_sent', 'timestamp']]
julia.dropna(inplace=True)
julia = julia[:200]
julia.head()

In [ ]:
juliaMessages = julia[julia.is_sent == 0][['text', 'timestamp']]
juliaMessages.sort_values(by='timestamp', inplace=True)

myMessages = julia[julia.is_sent == 1][['text', 'timestamp']]
myMessages.sort_values(by='timestamp', inplace=True)

In [ ]:
from pandas import DataFrame
from collections import deque

def displayChat(firstName:str, first:DataFrame, secondName:str, second:DataFrame):
    firstIter = first.iterrows()
    secondIter = second.iterrows()

    done = object()
    firstNext = next(firstIter, done)
    secondNext = next(secondIter, done)

    chat = deque()
    currentChater = firstName

    while firstNext is not done and secondNext is not done:
        firstTimestamp = firstNext[1].timestamp 
        secondTimestamp = secondNext[1].timestamp

        if firstTimestamp < secondTimestamp:
            if currentChater != firstName:
                chat.append("\n" + firstName + ":")
                currentChater = firstName
            chat.append(firstNext[1].text)
            firstNext = next(firstIter, done)
        else:
            if currentChater != secondName:
                chat.append("\n")
                chat.append((secondName + ":").rjust(75))
                currentChater = secondName
            chat.append(secondNext[1].text.rjust(75))
            secondNext = next(secondIter, done)

    for s in chat:
        print(s)
    

displayChat("Dima", myMessages, "Julia", juliaMessages)